In [1]:
# import utility libraries
from netCDF4 import Dataset
import pandas as pd
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
from mpl_toolkits.basemap import Basemap, cm
import cv2
%matplotlib inline

# import machine learning tools
from sklearn.preprocessing import StandardScaler
from keras.models import Sequential, Graph
from keras.layers import Dense, Flatten, Activation, Dropout
from keras.layers.convolutional import Convolution2D, ZeroPadding2D, MaxPooling2D
from keras.optimizers import SGD
from keras import backend as K
from sklearn.metrics import roc_auc_score, accuracy_score

from clustering import Location_Clusterer
from util import reformat_y, plot_list_in_2D



Using Theano backend.
Using gpu device 0: GRID K520 (CNMeM is disabled, cuDNN not available)


In [2]:
n = 8

In [3]:
folder = '/home/ubuntu/dataset/'
files = ['veg.nc', 'ppt.monthly.mask.nc', 'tmean.monthly.mask.nc', 'elev.nc']
var_names = ['Cv', 'Band1', 'Band1', 'elev']

lc = Location_Clusterer(n_clusters=n)
for f, var in zip(files, var_names):
    lc.read_data(folder+f, var)

lc.transform_data()
clusters = lc.fit_predict(lc.data2d)

#plot_list_in_2D(lc.coords2d[:,0], lc.coords2d[:,1], clusters)

In [4]:
# for i in xrange(n):
#     ind = (clusters==i)
#     plot_list_in_2D(lc.coords2d[:,0][ind], lc.coords2d[:,1][ind], clusters[ind])

In [9]:
%%writefile nn_input.py
# import utility libraries
from netCDF4 import Dataset
import numpy as np

class NN_Input(object):
    """
    Stores the input data ready for feeding into a Keras neural network. 

    To-Do:
    - add function to take the clustering data in some ways
    - add function to return the actual lat, lon, and time based on indices
    
    """
    def __init__(self, predict=2, history=2, box=5, random_seed=None, fill_value=-999):
        """
        Initialize a class for storing neural network input data. 
        
        Parameters
        ----------
        predict: int, number of time points ahead that the model will predict. 
                 For example, if predict=2, the model will predict 2 time points away from the given time. 
        history: int, number of time points for which data would be included as input.
                 For example, if data_length=3, the model will receive 3 time points worth of data (current time
                 point, the previous time point, and the timep point before that).
        """
        if random_seed is not None:
            np.random.seed(random_seed)
        self.lons = None
        self.lats = None
        self.times = None
        
        self.labels = None
        self.features = {}
        self.feature_types = {}
        self.variables = []
        self.fill_value = fill_value
        
        self.predict = predict
        self.history = history
        self.box = box
        
    def load_labels(self, f_path, var):
        """
        Load labels from netCDF file. 
        
        Parameters
        ----------
        f_path: string
        var: string
        """
        nc = Dataset(f_path, 'r')
        self.lons = nc.variables['lon'][:]
        self.lats = nc.variables['lat'][:]
        
        self.times = nc.variables['time'][self.history:-self.predict]
        n = self.predict + self.history
        self.labels = nc.variables[var][n:,:,:]
        
    def load_features(self, f_path, var, name, feature_type):
        """
        Load feature values from netCDF files. Stores feature type information. 
        
        Parameters
        ----------
        f_path: string, path to input netCDF file.
        var: string, variable name as appeared in the netCDF file. 
        name: string, name of the variable to be stored. 
        feature_type: string, must be one of the following: 'history_time_series', 'forecast_time_series', 
        'multi_layers', 'single_layer'
        """
        nc = Dataset(f_path, 'r')
        temp_data = nc.variables[var][:]
        
        # Storing information on whether the input features 
        self.feature_types[name] = feature_type
        self.variables.append(name)
        
        if self.feature_types[name] == 'history_time_series':
            self.features[name] = temp_data[:-self.predict, :, :]
        elif self.feature_types[name] == 'forecast_time_series':
            self.features[name] = temp_data[self.history:, :, :]
        else:
            self.features[name] = temp_data
        
    def get_batch(self, j, k):
        pass
    
    def get_features(self, i, j, k, ndim_out):
        """
        Given indices for latitude, longitude, and time point, returns the associated data from self.data. 
        
        Parameters
        ----------
        lat: int, index for the latitude desired. Must be within the range available in self.data. 
        lon: int, index for the longitude desired. Must be within the range available in self.data. 
        time: int, index for the time point desired. Must be within the range available in self.data. 
        """
        maps = None
#         lst = None
        for ix, feat in enumerate(self.variables):
            if self.feature_types[feat] == 'history_time_series':
                temp_data = self.features[feat][i:i+self.history+1, j-self.box:j+self.box+1, k-self.box:k+self.box+1]
            elif self.feature_types[feat] == 'forecast_time_series':
                temp_data = self.features[feat][i:i+self.predict+1, j-self.box:j+self.box+1, k-self.box:k+self.box+1]
#             elif self.feature_types[feat] == 'multi_layers':
#                 temp_data = self.features[feat][:, j, k].flatten()
#             else: 
#                 temp_data = self.features[feat][j, k]
            
            if len(temp_data.shape) == 3:                
                if np.sum(temp_data.mask) > len(temp_data.flatten())/2:
                    return None
                elif np.any(temp_data.mask):
                    temp_data = temp_data.filled(self.fill_value)
                    
                if maps is None:
                    maps = temp_data
                else:
                    if ndim_out == 4:
                        maps = np.ma.concatenate((maps, temp_data), axis=0)
                    elif ndim_out == 5:
                        maps = np.stack((maps, temp_data))
#             else:
#                 if lst is None:
#                     lst = temp_data
#                 else:
#                     lst = np.append(lst, temp_data)
#         return [maps, lst]
        return maps
        
    def select(self, n=None, t=None, cutoff=None, subset=None, ndim_out=4):
        """
        Selecting n data points randomly from the database before specified time cutoff. 
        
        Parameters
        ----------
        n: int, number of data points wanted. 
        cutoff: int, time cutoff for the training dataset. Default is half of the data available. 
        """
        if cutoff is None:
            cutoff = len(self.times)/2
        
        indices, labels, output_maps, output_lst = [], [], [], []
        
        mi = 0
        for ix, feat in enumerate(self.variables):
            if self.feature_types[feat] == 'history_time_series':
                mi += (self.history+1)
            elif self.feature_types[feat] == 'forecast_time_series':
                mi += (self.predict+1)
                
        if ndim_out == 4:
            map_dimensions = (mi, (2*self.box)+1, (2*self.box)+1)
        elif ndim_out == 5:
            map_dimensions = (len(self.variables), self.history+1, (2*self.box)+1, (2*self.box)+1)
        
        if n is None and t is None:
            for i in xrange(cutoff):
                for (k, j) in subset:
                    l = self.labels[i, j, k]
                    features = self.get_features(i, j, k, ndim_out=ndim_out)
                    if features is not None and l != np.nan and features.shape==map_dimensions:
                        indices.append([i, j, k])
                        labels.append(l)
                        output_maps.append(features)
#                         output_lst.append(features[1])
        
        elif t is not None:
            for (k, j) in subset:
                l = self.labels[t, j, k]
                features = self.get_features(t, j, k, ndim_out=ndim_out)
                if features is not None and l != np.nan and features.shape==map_dimensions:
                    indices.append([t, j, k])
                    labels.append(l)
                    output_maps.append(features)
#                     output_lst.append(features[1])
        
        else:
            while len(labels) < n:
                if subset is not None:
                    (k, j) = subset[np.random.choice(len(subset))]
                else:
                    j = np.random.randint(self.box, len(self.lats)-self.box)
                    k = np.random.randint(self.box, len(self.lons)-self.box)
                if cutoff < 0:
                    i = np.random.randint(cutoff, 0)
                else:
                    i = np.random.randint(cutoff)
                l = self.labels[i, j, k]
                features = self.get_features(i, j, k, ndim_out=ndim_out)
                if features is not None and l != np.nan and features.shape==map_dimensions:
                    indices.append([i, j, k])
                    labels.append(l)
                    output_maps.append(features)
#                     output_lst.append(features[1])
                    
        return np.array(indices), np.array(labels), np.array(output_maps)


Overwriting nn_input.py


In [6]:
from nn_input import NN_Input

nn = NN_Input(predict=2, history=5, box=5, random_seed=42)
nn.load_labels(folder+'sign.label.nc', 'Band1')

# f_paths = ['all.ndvi.nc','all.max.of.Wind.nc', 'all.min.of.Tmin.nc', 'all.mean.of.Tmin.nc', 'all.sum.of.Prec.nc',
#            'all.max.of.Tmax.nc', 'all.mean.of.Tmax.nc','elev.nc', 'veg.nc']
# variables = ['Band1', 'Wind', 'Tmin', 'Tmin', 'Prec', 'Tmax', 'Tmax', 'elev', 'Cv']
# names = ['ndvi', 'max_wind', 'min_tmin', 'mean_tmin', 'total_prec', 'max_tmax', 'mean_tmax', 'elev', 'veg']
# feature_types = ['history_time_series', 'forecast_time_series', 'forecast_time_series', 'forecast_time_series',
#                  'forecast_time_series', 'forecast_time_series', 'forecast_time_series',
#                 'single_layer', 'multi_layers']

f_paths = ['all.mean.of.Tmin.nc', 'all.mean.of.Tmin.nc']
variables = ['Tmin', 'Tmin']
names = ['mean_tmin_history', 'mean_tmin_forecast']
feature_types = ['history_time_series', 'forecast_time_series']

for f_path, v, n, feature_type in zip(f_paths, variables, names, feature_types):
    nn.load_features(folder+f_path, v, n, feature_type)


In [7]:
point = [308, 290]
for i, loc in enumerate(lc.ind2d):
    if loc[0] == point[0] and loc[1] == point[1]:
        cluster = clusters[i]

print 'Modeling for cluster', cluster
subset = lc.ind2d[clusters==cluster]

print 'Getting training dataset'
id_train, y_train, X_map_train = nn.select(n=100000, subset=subset)
y_train = reformat_y(y_train)

Modeling for cluster 1
Getting training dataset
(9, 11, 11)


AttributeError: 'NoneType' object has no attribute 'shape'

In [ ]:
print 'Getting test dataset'

t = (len(nn.times)/2) +1
print len(subset)
id_test, y_test, X_map_test = nn.select(t=t, subset=subset)
y_test = reformat_y(y_test)
print len(y_test)

In [ ]:
map_dimensions=X_map_train[0].shape
print map_dimensions
print np.all(X_map_train[0] == X_map_train[1])

mean_train = np.mean(X_map_train.flatten())
std_train = np.std(X_map_train.flatten())
X_map_train = (X_map_train-mean_train)/std_train
X_map_test = (X_map_test-mean_train)/std_train
print y_train.shape

In [ ]:
def build_base_sequential_NN(n_conv_layers=2, nb_filters=16, nb_conv=3, map_dimensions=None, nb_pool=2):

    model = Sequential()
    model.add(Convolution2D(nb_filters, nb_conv, nb_conv,
                            border_mode='valid',
                            input_shape=map_dimensions))
    
    model.add(Activation('relu'))
    model.add(Convolution2D(nb_filters, nb_conv, nb_conv))
    model.add(Activation('relu'))
    model.add(MaxPooling2D(pool_size=(nb_pool, nb_pool)))
    model.add(Dropout(0.25))

    model.add(Flatten())
    model.add(Dense(128))
    model.add(Activation('relu'))
    model.add(Dropout(0.5))
    model.add(Dense(2))
    model.add(Activation('softmax'))

    sgd = SGD()
    model.compile(loss='categorical_crossentropy',
                  optimizer=sgd,
                  metrics=['accuracy'])
    return model

model = build_base_sequential_NN(nb_filters=64, map_dimensions=map_dimensions)
model.fit(X_map_train, y_train, batch_size=50, nb_epoch=5, verbose=True, validation_data=(X_map_test, y_test))
train_predict = model.predict(X_map_train, verbose=True)
test_predict = model.predict(X_map_test, verbose=True)

In [ ]:
image_paths = []
for t in xrange(216, 221):

    id_test, y_test, X_map_test = nn.select(t=t, subset=subset)
    y_test = reformat_y(y_test)
    X_map_test = (X_map_test-mean_train)/std_train
    
    timestamp = nn.times[(id_test[0,0]).astype(int)]
    xs = nn.lons[(id_test[:,2]).astype(int)]
    ys = nn.lats[(id_test[:,1]).astype(int)]
    test_predict = model.predict(X_map_test)
    
    path = plot_compare_map(xs, ys, y_test[:,0], (test_predict[:,0]>threshold), 
                            timestamp, folder='/home/ubuntu/dataset/output/')
    image_paths.append(path)

In [ ]:
animate_maps(image_paths, '/home/ubuntu/dataset/output/test.mp4')